# Import

In [1]:
import pandas as pd
import numpy as np
import jsonlines
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext import data
from torchtext import datasets
from torchtext import vocab

In [2]:
# Directories & Paths
output_dir = 'data/emoji/'
EMOJI_MODEL_PATH = '/home/ben/data/acl_2020/emoji2vec.txt'
OUTPUT_DIR = '/home/ben/data/acl_2020/output/seewhence_emoji/'

# Load and split Sarcasm Data

In [3]:
# reddit_path = '/home/ben/data/acl_2020/reddit.jsonl'
# twitter_path = '/home/ben/data/acl_2020/twitter.jsonl'
# train_split = 0.8
# %run ./preprocessing.ipynb

In [4]:
%load_ext autoreload
%autoreload 2

from seewhence.emoji import *
from seewhence import models
from seewhence import train

# Torch Text Dataset

In [5]:
# Fields

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
EMOJI = data.Field()
LABEL = data.LabelField(dtype = torch.float)
fields = {'response': ('text', TEXT), 'label': ('label', LABEL), 'response_emoji_unicode': ('emoji', EMOJI)}

In [6]:
train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path = OUTPUT_DIR,
                                        train = 'train.csv',
                                        validation ='valid.csv',
                                        test = 'test.csv',
                                        format = 'csv',
                                        fields = fields
)

In [7]:
for t in train_data:
    if len(vars(t)['emoji']):
        print(vars(t))
        break

{'text': ['@USER', 'I', 'regret', 'everything', '🥴', '🥴', '🥴', 'I', '’', 'm', 'so', 'full', 'I', 'can', '’', 't', 'breathe'], 'label': '0', 'emoji': ['🥴', '🥴', '🥴']}


# Vectors

## Emoji Vectors

In [8]:
emoji_vectors = vocab.Vectors(EMOJI_MODEL_PATH)

## Text Vectors

In [9]:
text_vectors = vocab.GloVe(name='840B', dim=300)

## Build Vocab

In [10]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = text_vectors, 
                 unk_init = torch.Tensor.normal_)

In [11]:
EMOJI.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = emoji_vectors, 
                 unk_init = torch.Tensor.normal_)

In [12]:
LABEL.build_vocab(train_data)

# Iterator

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch = True,
    device = device)

# LSTM

In [14]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 128

emoji_vocab_size = len(EMOJI.vocab)
emoji_embedding_dim = emoji_vectors.dim

OUTPUT_DIM = 1
N_LAYERS = 5
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

output_path = '/models/emoji_lstm_'

model = models.EmojiLSTM(INPUT_DIM, 
            EMBEDDING_DIM,
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX,
            emoji_vocab_size,
            emoji_embedding_dim)



In [15]:
model.emoji_embedding.weight.data.copy_(EMOJI.vocab.vectors)
model.text_embedding.weight.data.copy_(TEXT.vocab.vectors)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.text_embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.text_embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

model.emoji_embedding.weight.data[UNK_IDX] = torch.zeros(emoji_embedding_dim)
model.emoji_embedding.weight.data[PAD_IDX] = torch.zeros(emoji_embedding_dim)

In [16]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [17]:
N_EPOCHS = 8

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train.train(model, train_iterator, optimizer, criterion, acc=train.f1_accuracy)
    valid_loss, valid_acc = train.evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = train.epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        if output_path:
            torch.save(model.state_dict(), output_path)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

NameError: name 'f1_score' is not defined

In [ ]:
model.load_state_dict(torch.load(output_path))
test_loss, test_acc = train.evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')